In [1]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using the GPU")
else:
    print("TensorFlow is NOT using the GPU")


2024-01-10 19:50:47.174351: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 19:50:47.608269: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-10 19:50:47.796046: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 19:50:49.450399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

TensorFlow is using the GPU


2024-01-10 19:50:51.882796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-10 19:50:52.297696: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-10 19:50:52.297752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


in command prompt base:
/mnt/c/Users/Ryan/Desktop/scouts

In [2]:
!pip install matplotlib


/bin/bash: /home/ryan/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
import tensorflow as tf

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()


In [5]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [6]:
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)


In [7]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [8]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

2024-01-10 19:50:56.777933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 19:50:56.780968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-10 19:50:56.781027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-10 19:50:56.781050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been bu

In [9]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [10]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [11]:
np.random.seed(42)
tf.random.set_seed(42)

In [12]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [13]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [14]:
dataset = dataset.prefetch(1)

In [15]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [17]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id, activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)


Epoch 1/10
31368/31368 [==============================] - 443s 14ms/step - loss: 1.6197
Epoch 2/10
31368/31368 [==============================] - 441s 14ms/step - loss: 1.5397
Epoch 3/10
31368/31368 [==============================] - 445s 14ms/step - loss: 1.5187
Epoch 4/10
31368/31368 [==============================] - 438s 14ms/step - loss: 1.5074
Epoch 5/10
31368/31368 [==============================] - 441s 14ms/step - loss: 1.5005
Epoch 6/10
31368/31368 [==============================] - 446s 14ms/step - loss: 1.4957
Epoch 7/10
31368/31368 [==============================] - 458s 15ms/step - loss: 1.4926
Epoch 8/10
31368/31368 [==============================] - 457s 14ms/step - loss: 1.4896
Epoch 9/10
31368/31368 [==============================] - 456s 14ms/step - loss: 1.4876
Epoch 10/10
31368/31368 [==============================] - 459s 15ms/step - loss: 1.4870


# Using the Model to Generate Text

In [18]:


def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)



In [19]:


X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char



'u'

In [20]:


tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()



array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

In [21]:


def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]



In [22]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

'u'

In [23]:


def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text



In [24]:


tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))



the maid in hell and be to hear
and so be to her th


In [25]:
print(complete_text("t", temperature=1))

to coop of signior coble country
to old aside. shal


In [26]:
print(complete_text("t", temperature=2))

tpeniom,
if you tigmor hichle'p on herit oul.-
apem


# Stateful RNN

In [27]:
tf.random.set_seed(42)

In [28]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [29]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [30]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])



In [31]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [32]:


model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])



Epoch 1/50
313/313 [==============================] - 7s 15ms/step - loss: 2.6187
Epoch 2/50
313/313 [==============================] - 4s 14ms/step - loss: 2.2387
Epoch 3/50
313/313 [==============================] - 4s 14ms/step - loss: 2.1076
Epoch 4/50
313/313 [==============================] - 4s 14ms/step - loss: 2.0317
Epoch 5/50
313/313 [==============================] - 4s 14ms/step - loss: 1.9821
Epoch 6/50
313/313 [==============================] - 4s 14ms/step - loss: 1.9441
Epoch 7/50
313/313 [==============================] - 4s 14ms/step - loss: 1.9164
Epoch 8/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8942
Epoch 9/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8773
Epoch 10/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8607
Epoch 11/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8495
Epoch 12/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8382
Epoch 13/50
3

# Using the Model to Generate Text


In [38]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [39]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

In [40]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [41]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

ValueError: Exception encountered when calling layer "sequential_2" "                 f"(type Sequential).

Input 0 of layer "gru_4" is incompatible with the layer: expected shape=(32, None, 39), found shape=(1, 10, 39)

Call arguments received by layer "sequential_2" "                 f"(type Sequential):
  • inputs=tf.Tensor(shape=(1, 10, 39), dtype=float32)
  • training=False
  • mask=None

In [42]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [43]:


tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

ValueError: Exception encountered when calling layer "sequential_2" "                 f"(type Sequential).

Input 0 of layer "gru_4" is incompatible with the layer: expected shape=(32, None, 39), found shape=(1, 1, 39)

Call arguments received by layer "sequential_2" "                 f"(type Sequential):
  • inputs=tf.Tensor(shape=(1, 1, 39), dtype=float32)
  • training=False
  • mask=None

In [44]:
print(complete_text("t", temperature=1))

ValueError: Exception encountered when calling layer "sequential_2" "                 f"(type Sequential).

Input 0 of layer "gru_4" is incompatible with the layer: expected shape=(32, None, 39), found shape=(1, 1, 39)

Call arguments received by layer "sequential_2" "                 f"(type Sequential):
  • inputs=tf.Tensor(shape=(1, 1, 39), dtype=float32)
  • training=False
  • mask=None

In [45]:
print(complete_text("t", temperature=2))

ValueError: Exception encountered when calling layer "sequential_2" "                 f"(type Sequential).

Input 0 of layer "gru_4" is incompatible with the layer: expected shape=(32, None, 39), found shape=(1, 1, 39)

Call arguments received by layer "sequential_2" "                 f"(type Sequential):
  • inputs=tf.Tensor(shape=(1, 1, 39), dtype=float32)
  • training=False
  • mask=None

# Stateful RNN


In [46]:
tf.random.set_seed(42)

In [47]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [48]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [50]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                        dropout=0.2,
                        batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                        dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                        activation="softmax"))
])


In [52]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [53]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 6s 15ms/step - loss: 2.6257
Epoch 2/50
313/313 [==============================] - 4s 14ms/step - loss: 2.2475
Epoch 3/50
313/313 [==============================] - 4s 14ms/step - loss: 2.1200
Epoch 4/50
313/313 [==============================] - 4s 14ms/step - loss: 2.0406
Epoch 5/50
313/313 [==============================] - 4s 14ms/step - loss: 1.9875
Epoch 6/50
313/313 [==============================] - 4s 14ms/step - loss: 1.9478
Epoch 7/50
313/313 [==============================] - 4s 14ms/step - loss: 1.9200
Epoch 8/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8972
Epoch 9/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8803
Epoch 10/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8642
Epoch 11/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8517
Epoch 12/50
313/313 [==============================] - 4s 14ms/step - loss: 1.8410
Epoch 13/50
3



To use the model with different batch sizes, we need to create a stateless copy. We can get rid of dropout since it is only used during training:


In [56]:
stateless_model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

To set the weights, we first need to build the model (so the weights get created):


In [57]:


stateless_model.build(tf.TensorShape([None, None, max_id]))



In [58]:


stateless_model.set_weights(model.get_weights())
model = stateless_model

In [59]:
tf.random.set_seed(42)

print(complete_text("t"))

ty:
what, dream'd, as it veny part beaugh too stand


# Sentiment Analysis


In [60]:


tf.random.set_seed(42)



In [63]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data()

17464789/17464789 [==============================] - 1s 0us/step


In [64]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [66]:
word_index = tf.keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1641221/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

In [70]:
!pip install --upgrade protobuf

/bin/bash: /home/ryan/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 3.1 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tb-nightly 2.16.0a20231226 requires google-auth-oauthlib<2,>=0.5, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tb-nightly 2.16.0a20231226 requires tensorboard-data-server<0.8.0,>=0.7.0, but you have tensorboard-data-server 0.6.1 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.2 which is incompatible.
tensorflow 2

In [72]:
!pip install --upgrade tensorflow-datasets

/bin/bash: /home/ryan/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.2
    Uninstalling protobuf-4.25.2:
      Successfully uninstalled protobuf-4.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tb-nightly 2.16.0a20231226 requires google-auth-oauthlib<2,>=0.5, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tb-nightly 2.16.0a20231226 requires tensorboard-data-server<0.8.0,>=0.7.0, but you have tensorboard-data-server 0.6.1 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.1

In [76]:
import tensorflow_datasets as tfds
from keras.datasets import imdb

datasets, info =  imdb.load_data("imdb_reviews", as_supervised=True, with_info=True)

TypeError: Unrecognized keyword arguments: {'as_supervised': True, 'with_info': True}.

In [68]:
datasets.keys()

AttributeError: 'list' object has no attribute 'keys'

In [ ]:


train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples



In [ ]:
train_size, test_size

In [ ]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

In [ ]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [ ]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))